# Libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras

Using TensorFlow backend.


# Data

In [2]:
dataset = pd.read_csv('featureset.csv')

In [3]:
testset = pd.read_csv('testset.csv')

In [4]:
dataset.head()

,URL,length,isIp,countAt,countDoubleSlash,countHyphen,countDots,countDelimeters,countSubDirectory,countQueries,domainRegistrationLength,alexaGlobalRank,bounceRate,dailyPageViewsPerVisitor,dailyTimeOnSite,searchVisits,totalSitesLinkingIn,label
0,https://pages.tmall.com,23,0,0,1,0,2,0,2,1,23,0.031250,-1.0,-1.0,-1,-1.0,-1.0,0
1,http://habrahabr.ru,19,0,0,1,0,1,0,2,1,12,-1.000000,-1.0,-1.0,-1,-1.0,-1.0,0
2,https://ebay.de,15,0,0,1,0,1,0,2,1,-1,0.007092,-1.0,-1.0,-1,-1.0,-1.0,0
3,http://account-chek-police.000webhostapp.com/p...,131,0,0,1,4,3,11,3,3,5,-1.000000,-1.0,-1.0,-1,-1.0,-1.0,1
4,http://aprecnotis.clan.su/recovery-login.htm,44,0,0,1,1,3,0,3,1,12,-1.000000,-1.0,-1.0,-1,-1.0,-1.0,1


In [5]:
testset.head()

,length,isIp,countAt,countDoubleSlash,countHyphen,countDots,countDelimeters,countSubDirectory,countQueries,domainRegistrationLength,alexaGlobalRank,bounceRate,dailyPageViewsPerVisitor,dailyTimeOnSite,searchVisits,totalSitesLinkingIn,label
0,39.0,0.0,0.0,1.0,0.0,2.0,0.0,2.0,1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
1,31.0,0.0,0.0,1.0,0.0,2.0,0.0,2.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
2,34.0,0.0,0.0,1.0,0.0,2.0,0.0,2.0,1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
3,37.0,0.0,0.0,1.0,0.0,2.0,0.0,2.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
4,33.0,0.0,0.0,1.0,0.0,2.0,0.0,2.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1


In [6]:
X = dataset.drop(['URL', 'label'], axis=1)
y = dataset['label']

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.2887, random_state=101)

In [9]:
X_test = testset.drop(['label'], axis=1)
y_test = testset['label']

# Scaled Features

In [10]:
from sklearn.preprocessing import MinMaxScaler

In [11]:
scaler = MinMaxScaler()
scaler.fit(X)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [12]:
Xs = scaler.transform(X)

In [13]:
Xs_train, Xs_eval, ys_train, ys_eval = train_test_split(Xs, y, test_size=0.2887, random_state=101)

In [14]:
Xs_test = scaler.transform(X_test)

In [15]:
ys_test = y_test

# Machine Learning

In [16]:
from sklearn.metrics import classification_report, confusion_matrix

### Neural Networks

In [17]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [18]:
def neuralnet():
    model = Sequential()
    model.add(Dense(76, activation='relu', input_dim=16))
    model.add(Dense(168, activation='relu'))
    model.add(Dense(368, activation='relu'))
    model.add(Dense(84, activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='rmsprop')
    return model

In [19]:
network = KerasClassifier(build_fn=neuralnet, batch_size=25, nb_epoch=100)

In [20]:
accuracies = cross_val_score(estimator=network, X=Xs, y=y, cv=10)
print("{} +- ({})".format(accuracies.mean(), accuracies.std()))

Epoch 1/1
590/590 [==============================] - 0s 543us/step
Epoch 1/1
590/590 [==============================] - 0s 414us/step
Epoch 1/1
590/590 [==============================] - 0s 496us/step
Epoch 1/1
590/590 [==============================] - 0s 610us/step
Epoch 1/1
589/589 [==============================] - 0s 652us/step
Epoch 1/1
589/589 [==============================] - 0s 838us/step
Epoch 1/1
589/589 [==============================] - 1s 955us/step
Epoch 1/1
589/589 [==============================] - 1s 957us/step
Epoch 1/1
589/589 [==============================] - 0s 674us/step
Epoch 1/1
589/589 [==============================] - 0s 822us/step
0.9397709380305621 +- (0.017254656927992156)


In [21]:
network = neuralnet()
network.fit(Xs, y, batch_size=25, nb_epoch=100)
y_pred = network.predict(Xs_test)

D:\Users\Ritvik\Anaconda3\envs\tfdeeplearning\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/100
5894/5894 [==============================] - 6s 1ms/step - loss: 0.1869 - acc: 0.9293
Epoch 2/100
5894/5894 [==============================] - 3s 485us/step - loss: 0.1383 - acc: 0.9433
Epoch 3/100
5894/5894 [==============================] - 4s 612us/step - loss: 0.1293 - acc: 0.9455
Epoch 4/100
5894/5894 [==============================] - 3s 578us/step - loss: 0.1258 - acc: 0.9462
Epoch 5/100
5894/5894 [==============================] - 3s 479us/step - loss: 0.1251 - acc: 0.9466
Epoch 6/100
5894/5894 [==============================] - 3s 590us/step - loss: 0.1213 - acc: 0.9489
Epoch 7/100
5894/5894 [==============================] - 4s 613us/step - loss: 0.1213 - acc: 0.9488
Epoch 8/100
5894/5894 [==============================] - 4s 643us/step - loss: 0.1215 - acc: 0.9496
Epoch 9/100
5894/5894 [==============================] - 4s 640us/step - loss: 0.1171 - acc: 0.9488
Epoch 10/100
5894/5894 [==============================] - 4s 639us/step - loss: 0.1151 - acc: 0.9518
E

5894/5894 [==============================] - 4s 675us/step - loss: 0.1318 - acc: 0.9523
Epoch 83/100
5894/5894 [==============================] - 4s 647us/step - loss: 0.1326 - acc: 0.9511
Epoch 84/100
5894/5894 [==============================] - 5s 764us/step - loss: 0.1335 - acc: 0.9530
Epoch 85/100
5894/5894 [==============================] - 4s 726us/step - loss: 0.1389 - acc: 0.9506
Epoch 86/100
5894/5894 [==============================] - 4s 691us/step - loss: 0.1216 - acc: 0.9515
Epoch 87/100
5894/5894 [==============================] - 4s 665us/step - loss: 0.1307 - acc: 0.9525
Epoch 88/100
5894/5894 [==============================] - 4s 655us/step - loss: 0.1376 - acc: 0.9547
Epoch 89/100
5894/5894 [==============================] - 4s 618us/step - loss: 0.1359 - acc: 0.9493
Epoch 90/100
5894/5894 [==============================] - 4s 636us/step - loss: 0.1242 - acc: 0.9527
Epoch 91/100
5894/5894 [==============================] - 4s 662us/step - loss: 0.1402 - acc: 0.9505
Epo

In [22]:
y_p = []
for y in y_pred:
    y_p.append(int(y.round()))

In [23]:
print(confusion_matrix(y_test, y_p))
print(classification_report(y_test, y_p))

[[49  0]
 [ 7 43]]
             precision    recall  f1-score   support

          0       0.88      1.00      0.93        49
          1       1.00      0.86      0.92        50

avg / total       0.94      0.93      0.93        99



# Model Pickeling

In [24]:
network.save('Keras.h5')

In [25]:
new_model = keras.models.load_model('Keras.h5')
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_61 (Dense)             (None, 76)                1292      
_________________________________________________________________
dense_62 (Dense)             (None, 168)               12936     
_________________________________________________________________
dense_63 (Dense)             (None, 368)               62192     
_________________________________________________________________
dense_64 (Dense)             (None, 84)                30996     
_________________________________________________________________
dense_65 (Dense)             (None, 20)                1700      
_________________________________________________________________
dense_66 (Dense)             (None, 1)                 21        
Total params: 109,137
Trainable params: 109,137
Non-trainable params: 0
_________________________________________________________________
